In [41]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches
import pandas as pd
import os

In [6]:
def load_data_from_file(filename):
    keypoints = []
    descriptors = []
    
    with open(filename, 'r') as file:
        lines = file.readlines()[2:]  
        
    for line in lines:
        numbers = list(map(float, line.split()))
        keypoints.append(numbers[0:5])
        descriptors.append(numbers[5:])
    
    keypoints = np.array(keypoints)
    descriptors = np.array(descriptors)
    
    return keypoints, descriptors

In [28]:
def get_ellipse_properties(keypoints):
    properties = []
    coord = []
    s = []
    o = []
    As = []
    for keypoint in keypoints:
        u, v, a, b, c = keypoint[:5]
        
        A = np.array([[a, b], [b, c]])

        eigenvalues, eigenvectors = np.linalg.eig(A)

        sorted_indices = np.argsort(eigenvalues)
        eigenvalues = eigenvalues[sorted_indices]
        eigenvectors = eigenvectors[:, sorted_indices]

        scale_si = np.sqrt(1/eigenvalues[0])

        orientation_oi = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]) * (180 / np.pi) 
        coord.append((u,v))
        s.append(scale_si)
        o.append(orientation_oi)
        As.append(A)
    return coord,s,o,As

In [57]:
keypoints_root = "data/keypoints"
images_root = "data/images"
images_path = sorted(os.listdir(images_root))
keypoints_path = sorted(os.listdir(keypoints_root))


In [58]:
path = os.path.join(keypoints_root, keypoints_path[0])
keypoints, descriptors = load_data_from_file(path)
coord,s,o,A = get_ellipse_properties(keypoints)
img_id = images_path[0].split('.')[0]
df = pd.DataFrame({
    'image_id': img_id,
    'coord': coord,
    'scale': s,
    'orientation': o,
    'matrix': A,
    'descriptor': list(descriptors)
})